In [2]:
import xarray as xr
import glob
import numpy as np

# Directory containing the NetCDF files
data_dir = "C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/"
file_pattern = f"{data_dir}*.nc"

# Step 1: Find and sort all NetCDF files
file_list = sorted(glob.glob(file_pattern))  # Sort to ensure chronological order
print(f"Found {len(file_list)} files: {file_list}")

# Define NAO region coordinates
lat_min, lat_max = 20.2617801, 80.57591623  # 20°N to 80°N
lon_min_360, lon_max_360 = 270, 40  # 90°W (270°E) to 40°E

# Step 2: Process each file
processed_datasets = []
for file in file_list:
    # Open the dataset
    ds = xr.open_dataset(file, decode_times=False)

    # Select the two parts of the longitude range
    lon_part1 = ds.sel(
        lat=slice(lat_min, lat_max),  # 20°N to 80°N
        lon=slice(0, lon_max_360)     # 0° to 40°E
    )
    lon_part2 = ds.sel(
        lat=slice(lat_min, lat_max),  # 20°N to 80°N
        lon=slice(lon_min_360, 358.75)  # 270°E to 358.75°
    )

    # Concatenate the two longitude parts
    nao_data = xr.concat([lon_part1, lon_part2], dim='lon')

    # Rename coordinates
    nao_data = nao_data.rename({'lon': 'longitude', 'lat': 'latitude'})

    # Decode the time coordinate
    nao_data['time'] = xr.decode_cf(nao_data)['time']

    # Select winter months (November, December, January, February, March)
    winter_months = [11, 12, 1, 2, 3]
    nao_data = nao_data.sel(time=nao_data['time'].dt.month.isin(winter_months))

    # Select only the required variables
    nao_data = nao_data[['time', 'longitude', 'latitude', 'PSL']]

    # Append to list of processed datasets
    processed_datasets.append(nao_data)


Found 17 files: ['C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1850-1859.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1860-1869.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1870-1879.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1880-1889.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1890-1899.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1900-1909.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1910-1919.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1920-1929.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1930-1939.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1940-1949.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1950-1959.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1960-1969.nc', 'C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data\\1970-1979.

In [4]:
# Step 3: Concatenate all processed datasets along the time dimension
merged_nao_data = xr.concat(processed_datasets, dim='time', data_vars='all', coords='different')


In [ ]:
# Sort by time to ensure chronological order
merged_nao_data = merged_nao_data.sortby('time')

# Step 4: Verify the merged dataset
print("Merged dataset shape:", merged_nao_data.dims)
print("Time range:", merged_nao_data['time'].min().values, "to", merged_nao_data['time'].max().values)
print("PSL shape:", merged_nao_data['PSL'].shape)

# Step 5: Save the merged dataset
output_file = "C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/merged_nao_data_1850-2049.nc"
merged_nao_data.to_netcdf(output_file)
print(f"Saved merged dataset to {output_file}")
# Step 6: Clean up
for ds in processed_datasets:
    ds.close()  # Close each dataset to free up resources

Merged dataset shape: FrozenMappingWarningOnValuesAccess({'time': 24916, 'latitude': 64, 'longitude': 105})
Time range: 1850-01-01 00:00:00 to 2015-01-01 00:00:00
PSL shape: (24916, 64, 105)
Saved merged dataset to C:/Users/31610/Documents/UvA/Master thesis/Data/CESM2 Data/merged_nao_data_1850-2049.nc


In [6]:
# Step 6: Clean up
for ds in processed_datasets:
    ds.close()  # Close each dataset to free up resources

In [10]:
print(merged_nao_data['PSL'])

<xarray.DataArray 'PSL' (time: 24916, latitude: 64, longitude: 105)> Size: 670MB
array([[[102037.44 , 102125.89 , 102234.7  , ..., 102002.55 ,
         101996.58 , 102015.61 ],
        [102155.78 , 102209.48 , 102299.75 , ..., 102091.375,
         102109.305, 102128.6  ],
        [102240.4  , 102250.89 , 102281.1  , ..., 102167.49 ,
         102199.13 , 102223.43 ],
        ...,
        [101868.375, 101857.945, 101863.54 , ..., 101920.89 ,
         101908.18 , 101887.586],
        [102194.95 , 102201.3  , 102203.016, ..., 102178.516,
         102185.34 , 102189.664],
        [102418.28 , 102414.875, 102410.66 , ..., 102435.39 ,
         102423.96 , 102419.48 ]],

       [[102037.43 , 102095.68 , 102204.96 , ..., 102013.52 ,
         102019.22 , 102018.   ],
        [102152.53 , 102192.37 , 102269.164, ..., 102107.81 ,
         102122.49 , 102135.18 ],
        [102238.96 , 102256.836, 102290.46 , ..., 102184.07 ,
         102214.52 , 102230.4  ],
...
        [ 99237.01 ,  99245.086,  99